In [ ]:
import io

import numpy as np
import torch
import tritonclient.http as httpclient
from PIL import Image

In [ ]:
import io
import os
import random
import numpy as np
import torch
import tritonclient.http as httpclient
from PIL import Image


def inference_with_bird_image(bird_species=None, split="test"):
    """
    Запуск инференса на Triton с реальным изображением птицы из датасета.
    
    Args:
        bird_species (str): Название вида птицы (например, "BALD EAGLE"). 
                           Если None - выберет случайный вид.
        split (str): Раздел датасета ("train", "val", "test")
    
    Returns:
        tuple: (predictions, true_species, image_path)
    """
    client = httpclient.InferenceServerClient(url="localhost:8900")

    # Путь к датасету
    data_dir = f"data/birds400/{split}"
    
    # Выбираем вид птицы
    if bird_species is None:
        # Случайный выбор вида
        species_dirs = [d for d in os.listdir(data_dir) 
                       if os.path.isdir(os.path.join(data_dir, d))]
        bird_species = random.choice(species_dirs)
    
    species_path = os.path.join(data_dir, bird_species)
    
    if not os.path.exists(species_path):
        print(f"Вид птицы '{bird_species}' не найден в {split} наборе")
        return None, None, None
    
    # Выбираем случайное изображение вида
    image_files = [f for f in os.listdir(species_path) 
                  if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not image_files:
        print(f"Изображения для вида '{bird_species}' не найдены")
        return None, None, None
    
    image_file = random.choice(image_files)
    image_path = os.path.join(species_path, image_file)
    
    print(f"🐦 Выбран вид птицы: {bird_species}")
    print(f"📸 Изображение: {image_path}")
    
    # Загружаем и подготавливаем изображение
    try:
        pil_image = Image.open(image_path).convert("RGB")
        
        # Конвертируем в байты (как в оригинальной функции)
        buffer = io.BytesIO()
        pil_image.save(buffer, format="JPEG")
        image_bytes = buffer.getvalue()
        image_input = np.frombuffer(image_bytes, dtype=np.uint8)
        
        print(f" Размер изображения в байтах: {image_input.shape[0]}")
        print(f" Формат: {pil_image.size} (ширина x высота)")
        
    except Exception as e:
        print(f" Ошибка загрузки изображения {image_path}: {e}")
        return None, None, None

    try:
        inputs = []
        batch_size = 1
        batched_input = image_input.reshape(1, -1)

        inputs.append(httpclient.InferInput("raw_image", batched_input.shape, "UINT8"))
        inputs[0].set_data_from_numpy(batched_input)

        outputs = []
        outputs.append(httpclient.InferRequestedOutput("predictions"))

        print(f" Отправка на Triton: форма={batched_input.shape}, batch_size={batch_size}")
        print(" Выполнение инференса...")

        response = client.infer(model_name="ensemble_trt", inputs=inputs, outputs=outputs)

        predictions = response.as_numpy("predictions")

        print(f" Предсказания получены!")
        print(f" Форма предсказаний: {predictions.shape}")
        print(f" Тип данных: {predictions.dtype}")
        
        # Получаем топ-5 предсказаний
        pred_flat = predictions.flatten()
        top5_indices = np.argsort(pred_flat)[-5:][::-1]
        top5_probs = pred_flat[top5_indices]
        
        return predictions, bird_species, image_path

    except Exception as e:
        print(f"❌ Ошибка инференса: {e}")
        return None, None, None

In [ ]:
# 1. Случайная птица из тестового набора
print("=== Случайная птица ===")
preds1, species1, path1 = inference_with_bird_image()

Creating random torch tensor with shape: (224, 224, 3)
Input image byte length: 30743
Data type: uint8
Value range: [0, 255]
Input shape being sent: (1, 30743)
Batch size: 1
Running inference...
Predictions shape: (1, 589)
Predictions dtype: float32
First 10 predictions: [-0.06658936 -0.01670837  0.02011108 -0.13464355 -0.05551147  0.00735474
  0.00852966 -0.02246094  0.05340576 -0.08770752]
Last 10 predictions: [ 0.07836914  0.07098389  0.16088867 -0.10070801  0.01976013 -0.03790283
 -0.01708984 -0.06860352  0.07781982  0.00190735]


In [52]:
probs.argmax(-1)

array([360])